**Import Modules**

In [1]:
import numpy as np
import h5py
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from keras.utils import to_categorical
from keras import regularizers

Using TensorFlow backend.


In [2]:
def load_dataset():
    train_dataset = h5py.File('datasets/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])
    train_set_y_orig = np.array(train_adataset["train_set_y"][:])

    test_dataset = h5py.File('datasets/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])
    test_set_y_orig = np.array(test_dataset["test_set_y"][:])

    classes = np.array(test_dataset["list_classes"][:])
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [3]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape
Y_train = to_categorical(Y_train_orig, num_classes=len(classes), dtype='float32').reshape((X_train.shape[0],len(classes)))
Y_test = to_categorical(Y_test_orig, num_classes=len(classes), dtype='float32').reshape((X_test.shape[0],len(classes)))


In [4]:
def signClassifierModel(input_shape):
    X_input=Input(input_shape)
    X=Conv2D(8,(4,4),strides=(1,1),padding='same',name='conv0',kernel_regularizer=regularizers.l2(0.01))(X_input)
    X=BatchNormalization(axis=3,name='bn0')(X)
    X=Activation('relu')(X)
    X=MaxPooling2D((8,8),name='max_pool0',padding='same')(X)
    X=Conv2D(16,(2,2),strides=(1,1),padding='same',name='conv1',kernel_regularizer=regularizers.l2(0.01))(X)
    X=BatchNormalization(axis=3,name='bn1')(X)
    X=Activation('relu')(X)
    X=MaxPooling2D((4,4),name='max_pool1',padding='same')(X)
    X=Flatten()(X)
    X=Dense(6,activation='softmax')(X)
    model=Model(inputs = X_input, outputs = X, name='signClassifier')
    
    return model

In [5]:
model = signClassifierModel((64,64,3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 64, 64, 8)         392       
_________________________________________________________________
bn0 (BatchNormalization)     (None, 64, 64, 8)         32        
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 8)         0         
_________________________________________________________________
max_pool0 (MaxPooling2D)     (None, 8, 8, 8)           0         
_________________________________________________________________
conv1 (Conv2D)               (None, 8, 8, 16)          528       
_________________________________________________________________
bn1 (BatchNormalization)     (None, 8, 8, 16)          64        
__________

In [6]:
model.compile(optimizer='adam',metrics=['accuracy'],loss='categorical_crossentropy')

In [7]:
model.fit(x=X_train, y=Y_train , epochs=300, batch_size=64)

Epoch 1/300
1080/1080 [==============================] - 3s 3ms/step - loss: 2.3514 - acc: 0.1852
Epoch 2/300
1080/1080 [==============================] - 0s 391us/step - loss: 1.9608 - acc: 0.2157
Epoch 3/300
1080/1080 [==============================] - 0s 368us/step - loss: 1.8383 - acc: 0.3019
Epoch 4/300
1080/1080 [==============================] - 0s 399us/step - loss: 1.7629 - acc: 0.3583
Epoch 5/300
1080/1080 [==============================] - 0s 348us/step - loss: 1.6782 - acc: 0.3963
Epoch 6/300
1080/1080 [==============================] - 0s 392us/step - loss: 1.6059 - acc: 0.4546
Epoch 7/300
1080/1080 [==============================] - 0s 339us/step - loss: 1.5339 - acc: 0.4833
Epoch 8/300
1080/1080 [==============================] - 0s 302us/step - loss: 1.4578 - acc: 0.5185
Epoch 9/300
1080/1080 [==============================] - 0s 393us/step - loss: 1.3852 - acc: 0.5620
Epoch 10/300
1080/1080 [==============================] - 0s 361us/step - loss: 1.3130 - acc: 0.5963
E

1080/1080 [==============================] - 0s 371us/step - loss: 0.0588 - acc: 0.9991
Epoch 162/300
1080/1080 [==============================] - 0s 365us/step - loss: 0.0575 - acc: 1.0000
Epoch 163/300
1080/1080 [==============================] - 0s 441us/step - loss: 0.0569 - acc: 1.0000
Epoch 164/300
1080/1080 [==============================] - 0s 410us/step - loss: 0.0575 - acc: 1.0000
Epoch 165/300
1080/1080 [==============================] - 0s 395us/step - loss: 0.0587 - acc: 1.0000
Epoch 166/300
1080/1080 [==============================] - 0s 437us/step - loss: 0.0555 - acc: 1.0000
Epoch 167/300
1080/1080 [==============================] - 0s 338us/step - loss: 0.0571 - acc: 1.0000
Epoch 168/300
1080/1080 [==============================] - 0s 394us/step - loss: 0.0551 - acc: 1.0000
Epoch 169/300
1080/1080 [==============================] - 0s 418us/step - loss: 0.0537 - acc: 1.0000
Epoch 170/300
1080/1080 [==============================] - 0s 364us/step - loss: 0.0569 - acc: 0

In [8]:
preds = model.evaluate(x=X_test,y=Y_test)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

120/120 [==============================] - 0s 1ms/step

Loss = 0.30723325510819754
Test Accuracy = 0.9083333293596904
